# Interpolation of audiometric BC direct thresholds to obtain 1/3 octave thresholds

5-6-2021 GC

In [1]:
from math import log10
from pathlib import Path
import numpy as np
import pandas as pd

In [2]:
def read_BC_direct():
    # read csv file with measured BC direct data
    f_in = '/media/guido/LACIE/Cingle_Guido/Master/Implant/Primary_data/BCdirect.csv'
    p_in = Path(f_in)   
    df = pd.read_csv(p_in, header=0, nrows=85, index_col='Study_ID')
    df = df.drop(['Unnamed: 0'], axis=1)
    d = {'BCdimp_250':'f_250_Hz', 'BCdimp_500':'f_500_Hz', 'BCdimp_750':'f_750_Hz', 
         'BCdimp_1000': 'f_1000_Hz', 'BCdimp_1500': 'f_1500_Hz', 'BCdimp_2000': 'f_2000_Hz', 
         'BCdimp_3000': 'f_3000_Hz', 'BCdimp_4000': 'f_4000_Hz', 'BCdimp_6000': 'f_6000_Hz'}
    df.rename(d, axis=1, inplace=True)
    # df.drop(['f_750_Hz', 'f_6000_Hz], axis=1, inplace=True)
    return df

In [3]:
def BAHA5P_forcelevels(df = read_BC_direct()):
    # make dataframe with RETFL value (to add to BC direct thresholds)
    idx = df.columns
    RETFL = [57.7, 52.7, 45.0, 39.6, 37.0, 28.9, 31.6, 32.1, 32.6]
    reference_force_levels = pd.Series(data=RETFL, index=idx, dtype='float32')
    return reference_force_levels

In [4]:
def thresholds_FL():
    # calculation of thresholds in Force Levels (correction with RETFL)
    df = read_BC_direct()
    corrected = df.add(BAHA5P_forcelevels(df), axis = 1)    # add Series with RETFL values = thresholds FL         
    corrected = corrected.round(decimals=1)
    return corrected

In [5]:
def band_names():
    # return list with column names for results table
    frequencies = [250, 315, 400, 500, 630, 800, 1000, 1250, 1600, 2000, 2500, 3150, 4000, 5000, 6300, 8000]
    l = list()
    for f in frequencies:
        st = 'f_' + str(f) + '_Hz'
        l.append(st)
    return l

In [6]:
def interpolation(fx, f1, f2, th1, th2):
    # function to interpolate threshold, frequency is log10 transformed
    xp = [log10(f1), log10(f2)]
    x = log10(fx)
    fp = [th1, th2]
    interpol = np.interp(x, xp, fp, left=None, right=None, period=None)
    return interpol

In [7]:
def interpoleted_columns():
    # interpolation matrix, and 'for' loop to traverse single column (frequencies)
    tfl = thresholds_FL()
    itpd = {315:[250, 500, 0, 1], 400:[250, 500, 0, 1], 630:[500, 750, 1, 2], 800:[750, 1000, 2, 3], 
            1250:[1000, 1500, 3, 4], 1600:[1500, 2000, 4, 5], 2500:[2000, 3000, 5, 6], 3150:[3000, 4000, 6, 7], 
            5000:[4000, 6000, 7, 8]}
    itpm = pd. DataFrame(itpd).T
    itpm.reset_index(inplace=True)
    itpm.rename({'index':'f'}, axis = 1, inplace = True)
    itpm = itpm.T
    d = dict()
    for i in range(len(itpm.T)):
        s = itpm[i]
        fx = s['f']
        f1 = s[0]
        f2 = s[1]
        y1 = tfl.iloc[:, s[2]]
        y2 = tfl.iloc[:, s[3]]
        dict_per_freq = interpolated(fx, f1, f2, y1, y2)
        d.update(dict_per_freq)
    df = pd.DataFrame(d, index=tfl.index, dtype='int')
    return df

In [8]:
def interpolated(fx, f1, f2, s1, s2):
    # apply interpolation to single threshold values and 
    #make a dictionary with result for all subjects for a single frequency
    tfl = thresholds_FL()
    l = list()
    for i in tfl.index:
        interpolated = interpolation(fx, f1, f2, s1[i], s2[i])
        interpolated = round(interpolated, 1)
        l.append(interpolated)
    name = 'f_' + str(fx) +'_Hz'
    dict_id_interp = {name:l}
    return(dict_id_interp)

In [9]:
def add_columns():
    # we suppose thresholds at 6.300 and 8 kHz equivalent at the 6 khz hreshod
    df = thresholds_FL()
    df['f_6300_Hz'] = df['f_6000_Hz']
    df['f_8000_Hz'] = df['f_6000_Hz']
    df.drop('f_6000_Hz', axis=1, inplace=True)
    return df

In [10]:
def write_res_in_master(df):
    # write results to csv file in Master directory
    fout = '/media/guido/LACIE/Cingle_Guido/Master/Implant/Calculated_data/BC_dir_thr_third.csv'
    pout = Path(fout)
    df.to_csv(pout)

In [11]:
def finalize_save_data():
    # function to join measured and interpolated data, and
    # to invoque the 'write' function
    intpld = interpoleted_columns()
    added = add_columns()
    complete = added.join(intpld)
    complete = complete[band_names()]
    write_res_in_master(complete)
    return complete

In [12]:
if __name__ == "__main__":
    result = finalize_save_data()

/tmp/ipykernel_6380/1344029667.py:21: FutureWarning: Could not cast to int64, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised.
  df = pd.DataFrame(d, index=tfl.index, dtype='int')


In [13]:
result

,f_250_Hz,f_315_Hz,f_400_Hz,f_500_Hz,f_630_Hz,f_800_Hz,f_1000_Hz,f_1250_Hz,f_1600_Hz,f_2000_Hz,f_2500_Hz,f_3150_Hz,f_4000_Hz,f_5000_Hz,f_6300_Hz,f_8000_Hz
Study_ID,,,,,,,,,,,,,,,,
34,82.7,79.4,75.9,72.7,65.5,54.3,34.6,38.7,42.4,43.9,48.1,51.7,52.1,66.1,77.6,77.6
35,75.7,74.7,73.7,72.7,65.5,58.8,54.6,50.4,50.8,63.9,54.4,49.2,62.1,73.4,82.6,82.6
43,72.7,71.0,69.3,67.7,63.3,57.7,49.6,48.2,47.4,48.9,53.1,60.1,77.1,60.9,47.6,47.6
47,82.7,77.7,72.5,67.7,69.0,65.4,49.6,45.4,40.2,33.9,35.4,37.5,42.1,42.4,42.6,42.6
50,82.7,79.4,75.9,72.7,65.5,56.5,44.6,43.2,40.2,33.9,40.9,47.5,52.1,49.6,47.6,47.6
53,82.7,81.0,79.3,77.7,67.6,57.7,49.6,42.7,37.4,38.9,45.9,50.8,47.1,52.9,57.6,57.6
54,82.7,79.4,75.9,72.7,65.5,59.9,59.6,49.9,42.4,43.9,48.1,52.5,57.1,51.9,47.6,47.6
56,82.7,82.7,82.7,82.7,69.8,62.2,69.6,62.7,56.3,53.9,60.9,69.2,82.1,60.4,42.6,42.6
59,90.7,86.4,81.9,77.7,76.2,72.7,64.6,52.2,40.2,33.9,40.9,47.5,52.1,49.6,47.6,47.6
